In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/yy2895/Desktop/update_stresult14-10-14.csv',header=None)
d = df.values
ntotal=len(df)

# we do not renormalize it 
#d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(7)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,8):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 14
n_encoder_h_2 = 9



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=7
training_epochs = 5000

In [2]:
#we store the variables here
log_files_path = 'C:/Users/yy2895/Desktop/st10-8-10/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [10, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 10], [10], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '8'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 10]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive200=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/yy2895/Desktop/st10-8-10/tmp/model.ckpt')
                        togive21=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,10)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive21.append(output_any_image)
                            
                        togive200=togive21
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 1.030205676
Validation Loss: 1.9615202
Epoch: 0002 cost = 1.013771960
Validation Loss: 1.4775621
Epoch: 0003 cost = 0.998120291
Validation Loss: 1.1968391
Epoch: 0004 cost = 0.985908134
Validation Loss: 1.0598452
Epoch: 0005 cost = 0.974218352
Validation Loss: 0.98892856
Epoch: 0006 cost = 0.961581443
Validation Loss: 0.9578355
Epoch: 0007 cost = 0.949166392
Validation Loss: 0.9166541
Epoch: 0008 cost = 0.937188370
Validation Loss: 0.8926748
Epoch: 0009 cost = 0.925271681
Validation Loss: 0.87541884
Epoch: 0010 cost = 0.913259149
Validation Loss: 0.8744597
Epoch: 0011 cost = 0.900952177
Validation Loss: 0.88079834
Epoch: 0012 cost = 0.888356635
Validation Loss: 0.8669518
Epoch: 0013 cost = 0.875756068
Validation Loss: 0.91058767
Epoch: 0014 cost = 0.863805192
Validation Loss: 0.9322156
Epoch: 0015 cost = 0.852942313
Validation Loss: 0.95171535
Epoch: 0016 cost = 0.842956041
Validation Loss: 0.96240824
Epoch: 0017 cost = 0.833492134
Validation Loss: 0.9524299

Epoch: 0146 cost = 0.215154761
Validation Loss: 0.21521723
Epoch: 0147 cost = 0.212372286
Validation Loss: 0.20976436
Epoch: 0148 cost = 0.209614766
Validation Loss: 0.19565614
Epoch: 0149 cost = 0.206882647
Validation Loss: 0.20294258
Epoch: 0150 cost = 0.204175304
Validation Loss: 0.20290299
Epoch: 0151 cost = 0.201492878
Validation Loss: 0.21035545
Epoch: 0152 cost = 0.198834581
Validation Loss: 0.19772717
Epoch: 0153 cost = 0.196200875
Validation Loss: 0.18325365
Epoch: 0154 cost = 0.193592067
Validation Loss: 0.18238586
Epoch: 0155 cost = 0.191006784
Validation Loss: 0.17286819
Epoch: 0156 cost = 0.188445721
Validation Loss: 0.16960235
Epoch: 0157 cost = 0.185908086
Validation Loss: 0.1694726
Epoch: 0158 cost = 0.183395456
Validation Loss: 0.16717397
Epoch: 0159 cost = 0.180905344
Validation Loss: 0.16542873
Epoch: 0160 cost = 0.178437846
Validation Loss: 0.16246752
Epoch: 0161 cost = 0.175995203
Validation Loss: 0.16145445
Epoch: 0162 cost = 0.173574656
Validation Loss: 0.1601356

Validation Loss: 0.04734453
Epoch: 0287 cost = 0.012487080
Validation Loss: 0.14038433
Epoch: 0288 cost = 0.012380270
Validation Loss: 0.1274511
Epoch: 0289 cost = 0.012284030
Validation Loss: 0.12652822
Epoch: 0290 cost = 0.012197687
Validation Loss: 0.14062563
Epoch: 0291 cost = 0.012119894
Validation Loss: 0.103797875
Epoch: 0292 cost = 0.012049943
Validation Loss: 0.14320879
Epoch: 0293 cost = 0.011987441
Validation Loss: 0.10678158
Epoch: 0294 cost = 0.011931716
Validation Loss: 0.08421671
Epoch: 0295 cost = 0.011882249
Validation Loss: 0.08693543
Epoch: 0296 cost = 0.011838373
Validation Loss: 0.0524057
Epoch: 0297 cost = 0.011800095
Validation Loss: 0.03015947
Epoch: 0298 cost = 0.011766929
Validation Loss: 0.08425004
Epoch: 0299 cost = 0.011739278
Validation Loss: 0.12620682
Epoch: 0300 cost = 0.011717433
Validation Loss: 0.09634807
Epoch: 0301 cost = 0.011701069
Validation Loss: 0.05187439
Epoch: 0302 cost = 0.011691683
Validation Loss: 0.06769596
Epoch: 0303 cost = 0.01168949

Epoch: 0428 cost = 0.010457646
Validation Loss: 0.020754427
Epoch: 0429 cost = 0.010453013
Validation Loss: 0.03520706
Epoch: 0430 cost = 0.010448540
Validation Loss: 0.03891751
Epoch: 0431 cost = 0.010443958
Validation Loss: 0.035102513
Epoch: 0432 cost = 0.010439667
Validation Loss: 0.04791942
Epoch: 0433 cost = 0.010435240
Validation Loss: 0.05951896
Epoch: 0434 cost = 0.010430504
Validation Loss: 0.03455893
Epoch: 0435 cost = 0.010425952
Validation Loss: 0.031835772
Epoch: 0436 cost = 0.010421469
Validation Loss: 0.034703407
Epoch: 0437 cost = 0.010417265
Validation Loss: 0.03665014
Epoch: 0438 cost = 0.010412752
Validation Loss: 0.05338687
Epoch: 0439 cost = 0.010408336
Validation Loss: 0.08598616
Epoch: 0440 cost = 0.010404181
Validation Loss: 0.03964733
Epoch: 0441 cost = 0.010399942
Validation Loss: 0.045139264
Epoch: 0442 cost = 0.010395522
Validation Loss: 0.058719743
Epoch: 0443 cost = 0.010391282
Validation Loss: 0.031098124
Epoch: 0444 cost = 0.010386932
Validation Loss: 0

Validation Loss: 0.027886309
Epoch: 0572 cost = 0.009957011
Validation Loss: 0.03198109
Epoch: 0573 cost = 0.009954397
Validation Loss: 0.032874554
Epoch: 0574 cost = 0.009951576
Validation Loss: 0.0441195
Epoch: 0575 cost = 0.009948904
Validation Loss: 0.039207462
Epoch: 0576 cost = 0.009946058
Validation Loss: 0.024211733
Epoch: 0577 cost = 0.009943878
Validation Loss: 0.051093854
Epoch: 0578 cost = 0.009940810
Validation Loss: 0.082185365
Epoch: 0579 cost = 0.009938055
Validation Loss: 0.053557865
Epoch: 0580 cost = 0.009935449
Validation Loss: 0.030696325
Epoch: 0581 cost = 0.009932793
Validation Loss: 0.03817368
Epoch: 0582 cost = 0.009930223
Validation Loss: 0.06538327
Epoch: 0583 cost = 0.009927461
Validation Loss: 0.081603415
Epoch: 0584 cost = 0.009925288
Validation Loss: 0.05024014
Epoch: 0585 cost = 0.009922058
Validation Loss: 0.03055882
Epoch: 0586 cost = 0.009919952
Validation Loss: 0.060816716
Epoch: 0587 cost = 0.009917031
Validation Loss: 0.039120067
Epoch: 0588 cost =

...

Validation Loss: 0.01611457
Epoch: 4287 cost = 0.008061261
Validation Loss: 0.015436144
Epoch: 4288 cost = 0.008036424
Validation Loss: 0.014546669
Epoch: 4289 cost = 0.008061286
Validation Loss: 0.031410236
Epoch: 4290 cost = 0.008036476
Validation Loss: 0.020377208
Epoch: 4291 cost = 0.008061439
Validation Loss: 0.022727594
Epoch: 4292 cost = 0.008036374
Validation Loss: 0.01089192
Epoch: 4293 cost = 0.008061421
Validation Loss: 0.022602934
Epoch: 4294 cost = 0.008036404
Validation Loss: 0.017134884
Epoch: 4295 cost = 0.008061597
Validation Loss: 0.024496911
Epoch: 4296 cost = 0.008036380
Validation Loss: 0.016599018
Epoch: 4297 cost = 0.008061598
Validation Loss: 0.0220907
Epoch: 4298 cost = 0.008036229
Validation Loss: 0.016956761
Epoch: 4299 cost = 0.008061669
Validation Loss: 0.02501163
Epoch: 4300 cost = 0.008036313
Validation Loss: 0.011099736
Epoch: 4301 cost = 0.008061763
Validation Loss: 0.028164934
Epoch: 4302 cost = 0.008036251
Validation Loss: 0.009101258
Epoch: 4303 cost

Validation Loss: 0.022081131
Epoch: 4439 cost = 0.008042445
Validation Loss: 0.019930024
Epoch: 4440 cost = 0.008020668
Validation Loss: 0.016032234
Epoch: 4441 cost = 0.008042383
Validation Loss: 0.01440444
Epoch: 4442 cost = 0.008020588
Validation Loss: 0.023818193
Epoch: 4443 cost = 0.008042327
Validation Loss: 0.013022342
Epoch: 4444 cost = 0.008020470
Validation Loss: 0.016115675
Epoch: 4445 cost = 0.008042213
Validation Loss: 0.014963471
Epoch: 4446 cost = 0.008020385
Validation Loss: 0.01603364
Epoch: 4447 cost = 0.008042197
Validation Loss: 0.024399443
Epoch: 4448 cost = 0.008020195
Validation Loss: 0.010670254
Epoch: 4449 cost = 0.008042194
Validation Loss: 0.025947314
Epoch: 4450 cost = 0.008020053
Validation Loss: 0.01723086
Epoch: 4451 cost = 0.008042149
Validation Loss: 0.031805716
Epoch: 4452 cost = 0.008019907
Validation Loss: 0.021203693
Epoch: 4453 cost = 0.008042098
Validation Loss: 0.020696115
Epoch: 4454 cost = 0.008019835
Validation Loss: 0.0139593845
Epoch: 4455 c

Epoch: 4581 cost = 0.008029695
Validation Loss: 0.015044847
Epoch: 4582 cost = 0.008008400
Validation Loss: 0.015942896
Epoch: 4583 cost = 0.008029535
Validation Loss: 0.015011225
Epoch: 4584 cost = 0.008008229
Validation Loss: 0.021562867
Epoch: 4585 cost = 0.008029365
Validation Loss: 0.011477253
Epoch: 4586 cost = 0.008007984
Validation Loss: 0.024781095
Epoch: 4587 cost = 0.008029071
Validation Loss: 0.015214872
Epoch: 4588 cost = 0.008007841
Validation Loss: 0.023324411
Epoch: 4589 cost = 0.008028768
Validation Loss: 0.017038643
Epoch: 4590 cost = 0.008007703
Validation Loss: 0.009302112
Epoch: 4591 cost = 0.008028704
Validation Loss: 0.03423453
Epoch: 4592 cost = 0.008007514
Validation Loss: 0.023452237
Epoch: 4593 cost = 0.008028463
Validation Loss: 0.03332548
Epoch: 4594 cost = 0.008007365
Validation Loss: 0.019691534
Epoch: 4595 cost = 0.008028308
Validation Loss: 0.028861487
Epoch: 4596 cost = 0.008007125
Validation Loss: 0.014538955
Epoch: 4597 cost = 0.008027991
Validation 

Validation Loss: 0.0126280105
Epoch: 4736 cost = 0.007995671
Validation Loss: 0.01669842
Epoch: 4737 cost = 0.008016546
Validation Loss: 0.021378227
Epoch: 4738 cost = 0.007995463
Validation Loss: 0.02835875
Epoch: 4739 cost = 0.008016397
Validation Loss: 0.012997367
Epoch: 4740 cost = 0.007995271
Validation Loss: 0.025321173
Epoch: 4741 cost = 0.008015993
Validation Loss: 0.008849142
Epoch: 4742 cost = 0.007995161
Validation Loss: 0.01940299
Epoch: 4743 cost = 0.008015815
Validation Loss: 0.008269777
Epoch: 4744 cost = 0.007994994
Validation Loss: 0.013149364
Epoch: 4745 cost = 0.008015619
Validation Loss: 0.020325266
Epoch: 4746 cost = 0.007994752
Validation Loss: 0.01627948
Epoch: 4747 cost = 0.008015419
Validation Loss: 0.024118286
Epoch: 4748 cost = 0.007994567
Validation Loss: 0.017731795
Epoch: 4749 cost = 0.008015213
Validation Loss: 0.019287774
Epoch: 4750 cost = 0.007994513
Validation Loss: 0.012564172
Epoch: 4751 cost = 0.008015008
Validation Loss: 0.027659198
Epoch: 4752 co

Validation Loss: 0.012402397
Epoch: 4891 cost = 0.008003627
Validation Loss: 0.022980157
Epoch: 4892 cost = 0.007983185
Validation Loss: 0.015545893
Epoch: 4893 cost = 0.008003409
Validation Loss: 0.009692678
Epoch: 4894 cost = 0.007982917
Validation Loss: 0.018548684
Epoch: 4895 cost = 0.008003137
Validation Loss: 0.019982707
Epoch: 4896 cost = 0.007982755
Validation Loss: 0.010206461
Epoch: 4897 cost = 0.008002936
Validation Loss: 0.015013025
Epoch: 4898 cost = 0.007982618
Validation Loss: 0.01604823
Epoch: 4899 cost = 0.008002668
Validation Loss: 0.038179535
Epoch: 4900 cost = 0.007982441
Validation Loss: 0.02685853
Epoch: 4901 cost = 0.008002491
Validation Loss: 0.033411335
Epoch: 4902 cost = 0.007982235
Validation Loss: 0.019609382
Epoch: 4903 cost = 0.008002216
Validation Loss: 0.014403167
Epoch: 4904 cost = 0.007982036
Validation Loss: 0.020090356
Epoch: 4905 cost = 0.008002125
Validation Loss: 0.01460402
Epoch: 4906 cost = 0.007981865
Validation Loss: 0.0194233
Epoch: 4907 cost

In [3]:
currentmin

0.0051956275

In [4]:
import csv
with open('C:/Users/yy2895/Desktop/update_stresult10-8-10.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive200)):
        wr.writerow(list(togive200[i][0]))

In [5]:
togive200

[array([[0.30135277, 0.6407913 , 0.31953076, 0.5442671 , 0.32541573,
         0.6369478 , 0.65868586, 0.19007502]], dtype=float32),
 array([[0.2868719 , 0.62144434, 0.35372093, 0.5702458 , 0.33163598,
         0.5996797 , 0.66369164, 0.19419207]], dtype=float32),
 array([[0.30408016, 0.6309211 , 0.37117013, 0.5706064 , 0.3490048 ,
         0.580057  , 0.62225336, 0.22141485]], dtype=float32),
 array([[0.31300646, 0.65416867, 0.35465544, 0.5461614 , 0.37911916,
         0.6098345 , 0.54796207, 0.24933141]], dtype=float32),
 array([[0.33119994, 0.68431985, 0.3367319 , 0.5219295 , 0.41427642,
         0.63560253, 0.45946386, 0.30158803]], dtype=float32),
 array([[0.30921236, 0.67877984, 0.3638485 , 0.53875715, 0.4008881 ,
         0.60477614, 0.51579225, 0.268495  ]], dtype=float32),
 array([[0.33112237, 0.6711122 , 0.34776714, 0.54532737, 0.41501915,
         0.6235803 , 0.47577155, 0.30650875]], dtype=float32),
 array([[0.30655482, 0.69062763, 0.32867515, 0.54684025, 0.42796287,
       